In [ ]:
!pip install datasets
!pip install timm

In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import io
import os
from sklearn.preprocessing import MultiLabelBinarizer


transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


csv_path = 'archive/Data_Entry_2017.csv'
df = pd.read_csv(csv_path)

# Separate the labels for each disease and generate single-label samples
expanded_rows = []
for _, row in df.iterrows():
    labels = row['Finding Labels'].split('|')
    for label in labels:
        new_row = row.copy()
        new_row['Finding Labels'] = label
        expanded_rows.append(new_row)

# Create a new DataFrame where each example has only one label
expanded_df = pd.DataFrame(expanded_rows)

all_labels = expanded_df['Finding Labels'].unique()
label_map = {label: idx for idx, label in enumerate(all_labels)}

# Convert each label to integer encoding
expanded_df['Encoded Labels'] = expanded_df['Finding Labels'].map(label_map)

class ChestXRayDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.data = dataframe
        self.img_dir = img_dir
        self.transform = transform
        
        self.image_paths = {}
        for root, _, files in os.walk(self.img_dir):
            for file in files:
                if file.endswith('.png'):
                    self.image_paths[file] = os.path.join(root, file)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]['Image Index']
        
        img_path = self.image_paths.get(img_name)
        if img_path is None:
            raise FileNotFoundError(f"Image {img_name} not found in specified directories.")
        
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        label = self.data.iloc[idx]['Encoded Labels']
        
        return image, label

img_dir = 'archive/images_all/'
dataset = ChestXRayDataset(dataframe=expanded_df, img_dir=img_dir, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

total_samples = len(data_loader.dataset)
print(f"Total samples in the dataset: {total_samples}")

# Get label distribution statistics
label_distribution = expanded_df['Finding Labels'].value_counts().to_dict()
print(f"Label distribution: {label_distribution}")

Total samples in the dataset: 141537
Label distribution: {'No Finding': 60361, 'Infiltration': 19894, 'Effusion': 13317, 'Atelectasis': 11559, 'Nodule': 6331, 'Mass': 5782, 'Pneumothorax': 5302, 'Consolidation': 4667, 'Pleural_Thickening': 3385, 'Cardiomegaly': 2776, 'Emphysema': 2516, 'Edema': 2303, 'Fibrosis': 1686, 'Pneumonia': 1431, 'Hernia': 227}


In [3]:
import torch
import timm
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Loading the DINO model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('vit_small_patch16_224_dino', pretrained=True)
model.to(device)
model.eval()


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity(

In [4]:
from tqdm import tqdm

def extract_features(backbone, dataloader, device):
    features, labels = [], []
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting features"):
            inputs, batch_labels = batch

            inputs = inputs.to(device)
            batch_labels = batch_labels.to(device)

            outputs = backbone(inputs)
            features.extend(outputs.cpu().numpy())
            labels.extend(batch_labels.cpu().numpy())
    
    return features, labels


# Extracting features using the DINOv2 model
train_features, train_labels = extract_features(model, data_loader, device)


print(f"Extracted feature shape: {np.array(train_features).shape}")
print(f"Extracted labels shape: {np.array(train_labels).shape}")


Extracting features: 100%|█████████████████████████████████████████████████████████| 4424/4424 [47:13<00:00,  1.56it/s]


Extracted feature shape: (141537, 384)
Extracted labels shape: (141537,)


In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

In [39]:

train_features_np = np.array(train_features)

n_samples = min(len(train_features_np), len(train_labels))
train_features_np = train_features_np[:n_samples]
train_labels_np = np.array(train_labels[:n_samples])

train_features_flatten = train_features_np.reshape(n_samples, -1)
input_dim = train_features_flatten.shape[1]
print(input_dim)
num_classes = np.max(train_labels_np) + 1
print(f"Number of classes (num_classes): {num_classes}")




384
Number of classes (num_classes): 15


In [40]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset


In [41]:

# Define the fully connected layer model
class FullyConnectedModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(FullyConnectedModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
            #nn.Softmax(dim=1)
        )

    def forward(self, x):
        return self.fc(x)

train_features_np, val_features_np, train_labels_np, val_labels_np = train_test_split(train_features_flatten, train_labels_np, test_size=0.1, random_state=42)

train_features_tensor = torch.tensor(train_features_np, dtype=torch.float32)
train_labels_tensor = torch.tensor(train_labels_np, dtype=torch.long)
val_features_tensor = torch.tensor(val_features_np, dtype=torch.float32)
val_labels_tensor = torch.tensor(val_labels_np, dtype=torch.long)

train_dataset = TensorDataset(train_features_tensor, train_labels_tensor)
val_dataset = TensorDataset(val_features_tensor, val_labels_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_dim = train_features_np.shape[1]
model = FullyConnectedModel(input_dim=input_dim, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 100
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    
    train_loader_tqdm = tqdm(train_loader, desc="Training", leave=False)
    for inputs, labels in train_loader_tqdm:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)

        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

print("Training Completed！")

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)

        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

y_pred = np.array(all_preds)
y_true = np.array(all_labels)


accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy:.4f}')


Epoch [1/100], Loss: 1.7863


Epoch [2/100], Loss: 1.7447


Epoch [3/100], Loss: 1.7343


Epoch [4/100], Loss: 1.7282


Epoch [5/100], Loss: 1.7238


...

Epoch [96/100], Loss: 1.6910


Epoch [97/100], Loss: 1.6906


Epoch [98/100], Loss: 1.6903


Epoch [99/100], Loss: 1.6912


Epoch [100/100], Loss: 1.6906
Training Completed！
Accuracy: 0.4584
